In [1]:
from scipy.spatial import ckdtree
import numpy as np
from tqdm import tqdm_notebook as tqdm
# %matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from Visual import animated_frames, save_frames

In [2]:
l = 1
d = l*2e-3
epsilon = d*2e-3

In [3]:
N = 200 #number of vertexes
Frames = 10 #number of frames between adding vercites
M = 100 #number of segment adding iteration between adding vertices
M = int(M/Frames)

In [18]:
def random_vertex():
    x, y = np.random.random(2)*2*l-l
    return np.array([x,y])

def find_segments(vertices, segments):
    active_segments = [] #segmenty które będą się rozrastać
    segments_vertices = [] #krawędzie do których będzie rozrastać się segment na odpowiednim miejscu powyżej
    tree = ckdtree.cKDTree(segments)
    for i in range(len(vertices)-1,-1,-1):
        dist, nearest_segments = tree.query(vertices[i])
        if dist < d:
            del vertices[i]
    for i in range(len(vertices)):
        dist, nearest_segments = tree.query(vertices[i],5)
        close = ((dist-dist[0])<epsilon)
        for s in nearest_segments[close]:
            if segments[s] in active_segments:
                index = active_segments.index(segments[s])
                segments_vertices[index].extend([i])
            else:
                active_segments.append(segments[s])
                segments_vertices.append([i])
    
    return active_segments, segments_vertices

def segment_adding(M:int,active_vertices,active_segments,segments_vertices,segments,visual):
    for m in range(M):
        for i in range(len(active_segments)-1, -1, -1):
            dist_x = 0
            dist_y = 0
            r_list = []
            for j in range(len(segments_vertices[i])):
                x = active_vertices[segments_vertices[i][j]][0] - active_segments[i][0]
                y = active_vertices[segments_vertices[i][j]][1] - active_segments[i][1]
                dist_x += x
                dist_y += y
                r_list.append((x**2 + y**2)**0.5)
            r = (dist_x**2 + dist_y**2)**0.5
            if r > d:
                new_seg = (active_segments[i][0]+dist_x*d/r,active_segments[i][1]+dist_y*d/r)
#                 print(((dist_x/r*d)**2+(dist_y/r*d)**2)**0.5)
                segments.append(new_seg)
                visual.append([active_segments[i],new_seg])
                active_segments[i] = new_seg
            else:
                closest_id = r_list.index(min(r_list))
                vertex = segments_vertices[i][closest_id]
                del segments_vertices[i][closest_id]
                if len(segments_vertices[i]) == 0:
                    del segments_vertices[i]
                    del active_segments[i]
                else:
                    x = active_vertices[vertex][0] - active_segments[i][0]
                    y = active_vertices[vertex][1] - active_segments[i][1]
                    r = r_list[closest_id]
                    new_seg = (active_segments[i][0]+x*d/r,active_segments[i][1]+y*d/r)
                    segments.append(new_seg)
                    visual.append([active_segments[i],new_seg])
                    active_segments.append(new_seg)
                    segments_vertices.append([vertex])
                    
#                         segments_vertices.remove(segments_vertices[i])
#                         active_segments.remove(active_segments[i])

In [19]:
def run(N=N,M=M,visualization=None):
    segments = [(0,0)]
    active_vertices = []
    visual = []
    animation_vertices = []
    animation_segments = []
    for n in tqdm(range(N)):
        active_vertices.append(random_vertex())
        active_segments, segments_vertices = find_segments(active_vertices, segments)
        for f in range(Frames):
            segment_adding(M,active_vertices,active_segments,segments_vertices,segments,visual)
            if len(active_segments) != 0:
                animation_vertices.append(np.array(active_vertices.copy()))
                animation_segments.append(np.array(visual.copy()))

    active_segments, segments_vertices = find_segments(active_vertices, segments)
    animation_vertices.append(np.array(active_vertices.copy()))
    animation_segments.append(np.array(visual.copy()))

    # active_vertices = np.vstack(active_vertices).T
    segments = np.vstack(np.array(segments)).T
    if visualization == 'end_picture':
        visual = np.array(visual).T
        return visual,active_vertices
    elif visualization == 'animation':
        return [animation_segments,animation_vertices]

In [20]:
animation_segments,animation_vertices = run(N,M,visualization='animation')
# visual,active_ver = run(N,M,visualization='end_picture')

In [21]:
# save_frames(animation_segments, animation_vertices,"multi_vertices_2")
# %matplotlib widget
animated_frames(animation_segments, animation_vertices)

FigureCanvasNbAgg()

IntSlider(value=0, max=899)

interactive(children=(Play(value=0, description='Press play', interval=200, max=899), Output()), _dom_classes=…

<function Functions.animated_frames.<locals>.update(f:int=0)>

In [9]:
# animation(animation_segments,animation_vertices,"anim")
# for i in animation_segments:
#     print(len(i))
# plt.plot(animation_vertices[0].T[0],animation_vertices.T[1],'o')
# plt.plot(animation_segments[0][0].T[0],animation_segments[0][0].T[1])
# animation_vertices
animation_segments[-1]

array([[[ 0.        ,  0.        ],
        [-0.00655748, -0.0075498 ]],

       [[-0.00655748, -0.0075498 ],
        [-0.01311496, -0.0150996 ]],

       [[-0.01311496, -0.0150996 ],
        [-0.01967244, -0.0226494 ]],

       ...,

       [[ 0.29774915,  0.63822203],
        [ 0.3048991 ,  0.63123073]],

       [[ 0.3048991 ,  0.63123073],
        [ 0.31204905,  0.62423944]],

       [[ 0.31204905,  0.62423944],
        [ 0.319199  ,  0.61724814]]])

In [10]:
fig = plt.figure(figsize=[15,15])
# plt.plot(segments[0],segments[1],'.')
plt.plot(visual[0],visual[1])
if active_vertices != []:
    plt.plot(active_vertices[0],active_vertices[1],'o')
# plt.savefig("takze_tego1.png")

FigureCanvasNbAgg()

NameError: name 'visual' is not defined

In [11]:
fig = plt.figure(figsize=[10,10])
vis = np.swapaxes(animation_segments[-10],0,2)
# plt.plot(visual[0],visual[1])
plt.plot(vis[0],vis[1])
print(visual.shape)

FigureCanvasNbAgg()

NameError: name 'visual' is not defined

In [52]:
def test_run(N=N,M=M,visualization=None):
    segments = [(0,0),(-d/np.sqrt(2),d/np.sqrt(2)),(d/np.sqrt(2),-d/np.sqrt(2))]
    active_vertices = [np.array([1,1])]
#     segments = [(0,0),(0,d),(0,-d)]
#     active_vertices = [np.array([1,0])]
    visual = []
    animation_vertices = []
    animation_segments = []
    active_segments, segments_vertices = find_segments(active_vertices, segments)
    for f in range(Frames):
        segment_adding(M,active_vertices,active_segments,segments_vertices,segments,visual)
        if len(active_segments) != 0:
            animation_vertices.append(np.array(active_vertices.copy()))
            animation_segments.append(np.array(visual.copy()))

    active_segments, segments_vertices = find_segments(active_vertices, segments)
    animation_vertices.append(np.array(active_vertices.copy()))
    animation_segments.append(np.array(visual.copy()))

    # active_vertices = np.vstack(active_vertices).T
    segments = np.vstack(np.array(segments)).T
    if visualization == 'end_picture':
        visual = np.array(visual).T
        return visual,active_vertices
    elif visualization == 'animation':
        return [animation_segments,animation_vertices]

In [53]:
animation_segments,animation_vertices = test_run(N,M,visualization='animation')

In [54]:
animated_frames(animation_segments, animation_vertices,"test")

FigureCanvasNbAgg()

IntSlider(value=0, max=10)

interactive(children=(Play(value=0, description='Press play', max=10), Output()), _dom_classes=('widget-intera…

<function Functions.animated_frames.<locals>.update(f:int=0)>